In [1]:
# imports

import os
import cv2
import sys
sys.path.append('..')
import numpy as np
from math import cos, sin
# from moviepy.editor import *
from lib.FSANET_model import *
import numpy as np
from keras.layers import Average
# from moviepy.editor import *
# from mtcnn.mtcnn import MTCNN
from time import perf_counter


Using TensorFlow backend.


In [2]:
def draw_axis(img_idx, img, yaw, pitch, roll, tdx=None, tdy=None, size = 50):
    txt_out.write(str(img_idx) + ' %f %f %f\n' % (yaw, pitch, roll))
    pitch = pitch * np.pi / 180
    yaw = -(yaw * np.pi / 180)
    roll = roll * np.pi / 180

    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img


In [3]:
def draw_results_ssd(img_idx, detected,input_img,faces,ad,img_size,img_w,img_h,model,time_detection,time_network,time_plot):

    # loop over the detections
    if detected.shape[2]>0:
        for i in range(0, detected.shape[2]):
            # extract the confidence (i.e., probability) associated with the
            # prediction
            confidence = detected[0, 0, i, 2]

            # filter out weak detections
            if confidence > 0.5:
                # compute the (x, y)-coordinates of the bounding box for
                # the face and extract the face ROI
                (h0, w0) = input_img.shape[:2]
                box = detected[0, 0, i, 3:7] * np.array([w0, h0, w0, h0])
                (startX, startY, endX, endY) = box.astype("int")
                # print((startX, startY, endX, endY))
                x1 = startX
                y1 = startY
                w = endX - startX
                h = endY - startY

                x2 = x1+w
                y2 = y1+h

                xw1 = max(int(x1 - ad * w), 0)
                yw1 = max(int(y1 - ad * h), 0)
                xw2 = min(int(x2 + ad * w), img_w - 1)
                yw2 = min(int(y2 + ad * h), img_h - 1)

                faces[i,:,:,:] = cv2.resize(input_img[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))
                faces[i,:,:,:] = cv2.normalize(faces[i,:,:,:], None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

                face = np.expand_dims(faces[i,:,:,:], axis=0)
                p_result = model.predict(face)

                face = face.squeeze()
                img = draw_axis(img_idx, input_img[yw1:yw2 + 1, xw1:xw2 + 1, :], p_result[0][0], p_result[0][1], p_result[0][2])

                input_img[yw1:yw2 + 1, xw1:xw2 + 1, :] = img

    cv2.imshow("result", input_img)

    return input_img #,time_network,time_plot

In [4]:
# custom variables
start = perf_counter()
video_path = "../../data/testvid_small.mp4"


In [5]:
# try:
#     os.mkdir('./img')
# except OSError:
#     print("Directory ./img already exists.")
#     pass

In [6]:
# load model and weights
img_size = 64
stage_num = [3,3,3]
lambda_local = 1
lambda_d = 1
img_idx = 0
detected = '' #make this not local variable
time_detection = 0
time_network = 0
time_plot = 0
skip_frame = 1 # original: 5; every 5 frame do 1 detection and network forward propagation
ad = 0.6

In [7]:
#Parameters
num_capsule = 3
dim_capsule = 16
routings = 2
stage_num = [3,3,3]
lambda_d = 1
num_classes = 3
image_size = 64
num_primcaps = 7*3
m_dim = 5
S_set = [num_capsule, dim_capsule, routings, num_primcaps, m_dim]

model1 = FSA_net_Capsule(image_size, num_classes, stage_num, lambda_d, S_set)()
model2 = FSA_net_Var_Capsule(image_size, num_classes, stage_num, lambda_d, S_set)()

num_primcaps = 8*8*3
S_set = [num_capsule, dim_capsule, routings, num_primcaps, m_dim]

model3 = FSA_net_noS_Capsule(image_size, num_classes, stage_num, lambda_d, S_set)()



Tensor("conv2d_7/Sigmoid:0", shape=(None, 8, 8, 1), dtype=float32)
Tensor("reshape_1/Reshape:0", shape=(None, None), dtype=float32)
Tensor("moments_layer_1/moments/Squeeze_1:0", shape=(None, 8, 8), dtype=float32)
Tensor("reshape_13/Reshape:0", shape=(None, None), dtype=float32)


In [8]:
# Load models
print('Loading models ...')

weight_file1 = '../pre-trained/300W_LP_models/fsanet_capsule_3_16_2_21_5/fsanet_capsule_3_16_2_21_5.h5'
model1.load_weights(weight_file1)
print('Finished loading model 1.')

weight_file2 = '../pre-trained/300W_LP_models/fsanet_var_capsule_3_16_2_21_5/fsanet_var_capsule_3_16_2_21_5.h5'
model2.load_weights(weight_file2)
print('Finished loading model 2.')

weight_file3 = '../pre-trained/300W_LP_models/fsanet_noS_capsule_3_16_2_192_5/fsanet_noS_capsule_3_16_2_192_5.h5'
model3.load_weights(weight_file3)
print('Finished loading model 3.')

inputs = Input(shape=(64,64,3))
x1 = model1(inputs) #1x1
x2 = model2(inputs) #var
x3 = model3(inputs) #w/o
avg_model = Average()([x1,x2,x3])
model = Model(inputs=inputs, outputs=avg_model)


Loading models ...
Finished loading model 1.
Finished loading model 2.
Finished loading model 3.


In [9]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
protoPath = os.path.sep.join(["face_detector", "deploy.prototxt"])
modelPath = os.path.sep.join(["face_detector",
    "res10_300x300_ssd_iter_140000.caffemodel"])
net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

[INFO] loading face detector...


In [10]:
# capture video
# cap = cv2.VideoCapture(0)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1024*1)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 768*1)

cap = cv2.VideoCapture(video_path)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1024*1)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 768*1)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # float
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # float

In [11]:
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out1 = cv2.VideoWriter('output/video/output-FSANET_ssd1.avi' , fourcc, 1, (width, height))
out30 = cv2.VideoWriter('output/video/output-FSANET_ssd30.avi', fourcc, 30, (width, height))
txt_out = open('output/video/output-FSANET_ssd.txt', 'w')

In [12]:
# Pose detection
print('Start detecting pose ...')
detected_pre = np.empty((1,1,1))

while True:
    # get video frame
    ret, input_img = cap.read()

    if not ret:
        cv2.destroyAllWindows()
        break

    img_idx = img_idx + 1
    img_h, img_w, _ = np.shape(input_img)

    if img_idx==1 or img_idx%skip_frame == 0:
        time_detection = 0
        time_network = 0
        time_plot = 0

        # detect faces using LBP detector
        gray_img = cv2.cvtColor(input_img,cv2.COLOR_BGR2GRAY)
        # detected = face_cascade.detectMultiScale(gray_img, 1.1)
        # detected = detector.detect_faces(input_img)
        # pass the blob through the network and obtain the detections and
        # predictions
        blob = cv2.dnn.blobFromImage(cv2.resize(input_img, (300, 300)), 1.0,
            (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detected = net.forward()

        if detected_pre.shape[2] > 0 and detected.shape[2] == 0:
            detected = detected_pre

        faces = np.empty((detected.shape[2], img_size, img_size, 3))

        input_img = draw_results_ssd(img_idx, detected,input_img,faces,ad,img_size,img_w,img_h,model,time_detection,time_network,time_plot)
        # cv2.imwrite('img/'+str(img_idx)+'.png',input_img)
        out1.write(input_img)
        out30.write(input_img)

    else:
        input_img = draw_results_ssd(img_idx, detected,input_img,faces,ad,img_size,img_w,img_h,model,time_detection,time_network,time_plot)


    if detected.shape[2] > detected_pre.shape[2] or img_idx%(skip_frame*3) == 0:
        detected_pre = detected

    key = cv2.waitKey(1)

txt_out.write("Runtime: " + str(perf_counter() - start) + "seconds")

out1.release()
out30.release()
cap.release()
txt_out.close()

Start detecting pose ...
